# Using machine learning to identity clusters of 'at risk' employees.

In [28]:
## Step 1 - Import necessary libraries
import pandas as pd
import sklearn
import joblib  # for saving an ml model

# Robust data-file finder and loader (uses a relative path for reading)
from pathlib import Path
import os


Identifying causes of attrition and enabling the business to identify different groups of 'at risk' employees is key. 

This notebook will create a model that will assess the 1400 rows of data currently available (split by 'train' and 'test' groups) to identify if there are logical groupings of exited staff, to enable future departures to be anticipated and, if desired, attempts made to retain.

### Step 1 - import the cleaned data

In [39]:
# Load cleaned data using an explicit relative path
df = pd.read_csv('../Data files/HR_Attrition_Cleaned.csv')

print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1470 non-null   int64  
 1   Attrition                 1470 non-null   object 
 2   BusinessTravel            1470 non-null   object 
 3   DailyRate                 1470 non-null   int64  
 4   Department                1470 non-null   object 
 5   DistanceFromHome          1470 non-null   int64  
 6   Education                 1470 non-null   object 
 7   EducationField            1470 non-null   object 
 8   EnvironmentSatisfaction   1470 non-null   object 
 9   Gender                    1470 non-null   object 
 10  HourlyRate                1470 non-null   int64  
 11  JobInvolvement            1470 non-null   object 
 12  JobLevel                  1470 non-null   int64  
 13  JobRole                   1470 non-null   object 
 14  JobSatis

Next, as we're working with a mix of numeric and string columns, we'll identify the data types in readiness to encoding. But our target column (Attrition) needs to be removed.

In [40]:
target_col = "Attrition"

numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

# Remove target from both lists if present
numeric_cols = [col for col in numeric_cols if col != target_col]
categorical_cols = [col for col in categorical_cols if col != target_col]

Now we need to split our data into a training set and then a testing set.

In [42]:
from sklearn.model_selection import train_test_split

X = df.drop(target_col, axis=1)
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (1029, 34) (1029,)
Test shape: (441, 34) (441,)


Now we need to pre-process our data to allow the pipeline to handle those different data types

In [43]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols)
])

Now the pipeline model can be built. We are using Random Forest because it handles mixed data sets, can identify complex interactions (ydata-profiling already identified that there's no single strong correlation for Attrition) and can rank importance. It is also more appropriate for 'imbalanced' data sets, like attrition, where values are more likely to be in the 'still employed' side of the data.

In [45]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", RandomForestClassifier(random_state=42))
])

Now we train the model using the 'train' data set

In [46]:
pipeline.fit(X_train, y_train)
print("Model score:", pipeline.score(X_test, y_test))

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values